In [64]:
import numpy as np
import pandas as pd
import json

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.keras.optimizers import Adam

In [65]:
dataframe = pd.read_csv('yorumlar.csv', sep = "\t")
target = dataframe['Puan'].values.tolist()
data = dataframe['Yorum'].values.tolist()

In [66]:
dataframe

,Puan,Yorum
0,1.0,Hızlı ve özenli kargo bu fiyata gayet iyi bir ...
1,1.0,"Harika bir kitqp,şiddetle tavsiye??"
2,1.0,Şahane bir plak ! Mermer baskısı o kadar şık o...
3,1.0,İnanılmaz keyifli bir oyun. Hem heyecan hem ke...
4,1.0,Alısverışten çok memnun kaldım eksıksız. Ç sor...
...,...,...
28978,1.0,yer kısıtı olanlar için çok kullanışlı bir ürü...
28979,0.0,içi boş bir kitap her sayfa birbirinden alakas...
28980,1.0,uygun fiyat
28981,1.0,Ürün elime ulaştı. Henüz okumadım ama firma ge...


In [67]:
cutoff = int(len(data) * 0.85)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [68]:
len(x_train)

27533

In [69]:
x_train[245]

'Dışı veya içi herşeyi mükemmel yeni yeni okumaya başladım fakat kalitesini anladım.'

In [70]:
with open('tokenizer.json') as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [71]:
json_tokenizer['güzel']

5

In [72]:
def tokenlestir(yorumListesi):
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in yorum.split():
            if (len(y_yorum) < 50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
                
        if (len(y_yorum) < 50):
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype = int))
            y_yorum = sifirlar + y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype = np.dtype(np.int32))

In [73]:
egitim_kume = tokenlestir(x_train)

In [74]:
egitim_kume[245]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,  383, 1101, 1557,   59,
         76,   76,  148,  305,   85,  991], dtype=int32)

In [75]:
test_kume = tokenlestir(x_test)

In [76]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=10001,
                    output_dim=embedding_size,
                    input_length=50,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 50, 50)            500050    
_________________________________________________________________
gru_9 (GRU)                  (None, 50, 16)            3216      
_________________________________________________________________
gru_10 (GRU)                 (None, 50, 8)             600       
_________________________________________________________________
gru_11 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 504,027
Trainable params: 504,027
Non-trainable params: 0
_________________________________________________________________


In [77]:
n_y_train = np.array(y_train)

In [78]:
model.fit(egitim_kume, n_y_train, epochs = 5, batch_size = 256)

Epoch 1/5
108/108 [==============================] - 18s 108ms/step - loss: 0.4914 - accuracy: 0.9122
Epoch 2/5
108/108 [==============================] - 12s 110ms/step - loss: 0.2336 - accuracy: 0.9316
Epoch 3/5
108/108 [==============================] - 12s 115ms/step - loss: 0.1614 - accuracy: 0.9502
Epoch 4/5
108/108 [==============================] - 11s 97ms/step - loss: 0.1238 - accuracy: 0.9651
Epoch 5/5
108/108 [==============================] - 11s 99ms/step - loss: 0.0935 - accuracy: 0.9767


In [79]:
n_y_test = np.array(y_test)

In [80]:
model.evaluate(test_kume, n_y_test)

46/46 [==============================] - 2s 23ms/step - loss: 0.0703 - accuracy: 0.9869


[0.07032211124897003, 0.9868965744972229]

In [81]:
model.save('25_mart_2021.h5')

In [82]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "gitar kursuna yeni başladım. ne almalıyım diye bakınırken Hepsiburada da bu ürünü gördüm. fiyatı ve hediyeleri ile birlikte çok iyi fiyat. siyah olmasıda benim için ayrı güzel."
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
yazilar = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [83]:
deneme_kume = tokenlestir(yazilar)

In [84]:
deneme_kume[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  152, 3872,   76,
         66,   86,   16,    6,   67,  127,    3, 4443,   33,  285,    2,
         11,  919, 3228,  150,    7,  247], dtype=int32)

In [85]:
sonuc = model.predict(deneme_kume)

In [86]:
sonuc

array([[0.98209476],
       [0.98137414],
       [0.12100825],
       [0.98195505],
       [0.22872421],
       [0.33732605],
       [0.9516531 ],
       [0.8757956 ],
       [0.9821422 ],
       [0.191906  ]], dtype=float32)